In [ ]:
import os
import joblib
from interpret import show

MODEL_DIR = "models"  # adjust if needed

def load_ebm(sex: str, variant: str, model_dir: str = MODEL_DIR):
    """
    Load a saved EBM model (possibly wrapped in a pipeline) and return
    the ExplainableBoostingClassifier object.
    """
    model_path = os.path.join(model_dir, f"ebm_{sex}_{variant}.pkl")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    obj = joblib.load(model_path)

    # Your saving format was {"model": pipeline_or_ebm, "features": [...]}
    if isinstance(obj, dict):
        pipe = obj["model"]
    else:
        pipe = obj

    # If it's an imblearn/sklearn pipeline with a "model" step, extract EBM
    if hasattr(pipe, "named_steps") and "model" in pipe.named_steps:
        ebm = pipe.named_steps["model"]
    else:
        ebm = pipe

    return ebm

In [ ]:
# Example: men, TN-PCA combined model
sex = "M"
variant = "tnpca"   # or "pca", "cog_only", "tnpca_only", etc.

ebm_model = load_ebm(sex, variant)
expl = ebm_model.explain_global(name=f"{sex}-{variant}")
show(expl)  # this is the interpret dashboard as notebook output
